In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence

In [ ]:
classifier = TextClassifier.load('en-sentiment')

2021-03-12 10:05:49,299 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmpfmpxfjtj


100%|██████████| 265512723/265512723 [00:31<00:00, 8437568.05B/s] 

2021-03-12 10:06:21,238 copying /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmpfmpxfjtj to cache at /Users/puneet/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-03-12 10:06:21,591 removing temp file /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmpfmpxfjtj
2021-03-12 10:06:21,611 loading file /Users/puneet/.flair/models/sentiment-en-mix-distillbert_4.pt


In [ ]:
sentence = Sentence('Flair is pretty neat!')
classifier.predict(sentence)

In [ ]:
# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

Sentence above is:  [POSITIVE (0.9997)]


# Training a custom text classifier

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")


In [ ]:
train_df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [ ]:
data = train_df.sample(frac=1).drop_duplicates()

data = data[['author','text']].rename(columns={'author':'label'})
data['label'] = '__label__'+data['label'].astype(str)

data.iloc[0:int(len(data)*0.8)].to_csv('temp/train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('temp/test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv('temp/dev.csv', sep='\t', index = False, header = False);

# Training a classifier

In [ ]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

In [ ]:
from flair.data import Corpus
from flair.datasets import ClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = './temp'

# load corpus containing training, test and dev data
corpus: Corpus = ClassificationCorpus(data_folder,
                                      test_file='test.csv',
                                      dev_file='dev.csv',
                                      train_file='train.csv')

2021-03-12 10:19:11,084 Reading data from temp
2021-03-12 10:19:11,085 Train: temp/train.csv
2021-03-12 10:19:11,085 Dev: temp/dev.csv
2021-03-12 10:19:11,086 Test: temp/test.csv


In [ ]:
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

2021-03-12 10:19:36,680 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmp4wj73ye6


100%|██████████| 160000128/160000128 [00:12<00:00, 12616893.60B/s]

2021-03-12 10:19:49,874 copying /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmp4wj73ye6 to cache at /Users/puneet/.flair/embeddings/glove.gensim.vectors.npy


2021-03-12 10:19:50,171 removing temp file /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmp4wj73ye6
2021-03-12 10:19:50,673 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmpls3ky6mi


100%|██████████| 21494764/21494764 [00:03<00:00, 6283289.68B/s]

...

100%|██████████| 19689779/19689779 [00:02<00:00, 7406853.36B/s]

2021-03-12 10:19:59,462 copying /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmpaanh816y to cache at /Users/puneet/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2021-03-12 10:19:59,493 removing temp file /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmpaanh816y


2021-03-12 10:20:00,006 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmpev5x6_01


100%|██████████| 19689779/19689779 [00:02<00:00, 8067511.30B/s]

2021-03-12 10:20:02,897 copying /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmpev5x6_01 to cache at /Users/puneet/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2021-03-12 10:20:02,930 removing temp file /var/folders/x5/5m2qt29d3493hj8vtxfdxs0h0000gn/T/tmpev5x6_01


In [ ]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

<ipython-input-19-a9bdbbe77cfc>:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)


In [ ]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

2021-03-12 10:20:17,602 Computing label dictionary. Progress:


100%|██████████| 17621/17621 [00:05<00:00, 3111.09it/s]

2021-03-12 10:20:40,802 [b'MWS', b'HPL', b'EAP']


In [ ]:
trainer = ModelTrainer(classifier, corpus)

In [ ]:
trainer.train('./', learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=10)

2021-03-12 10:21:12,547 ----------------------------------------------------------------------------------------------------
2021-03-12 10:21:12,548 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512)
 

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x145be58b0>
Traceback (most recent call last):
  File "/Users/puneet/.virtualenvs/torch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/Users/puneet/.virtualenvs/torch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/local/Cellar/python@3.9/3.9.2/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/local/Cellar/python@3.9/3.9.2/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/local/Cellar/python@3.9/3.9.2/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/connection.py", line 936, in wait
    ready = selector

2021-03-12 10:31:26,589 ----------------------------------------------------------------------------------------------------
2021-03-12 10:31:26,592 Exiting from training early.
2021-03-12 10:31:26,593 Saving model ...
2021-03-12 10:31:29,561 Done.
2021-03-12 10:31:29,568 ----------------------------------------------------------------------------------------------------
2021-03-12 10:31:29,569 Testing using best model ...


KeyboardInterrupt: 